In [1]:
import os
import requests
import json
import urllib3
from datetime import datetime, timedelta
from password_manager import PasswordManager
import modules.functions as fn

In [2]:
config_file = "config_encrypted.json"

In [ ]:
# functions to filter by date
def get_current_time():
    return datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')

In [4]:
def get_24_hours_ago():
    now = datetime.utcnow()
    twenty_four_hours_ago = now - timedelta(hours=24)
    return twenty_four_hours_ago.strftime("%Y-%m-%dT%H:%M:%SZ")

In [5]:
def get_filtered_results(url, headers, params, fields, cert_file):
    all_filtered_results = []
    page = 1
    total_pages = None

    while total_pages is None or page <= total_pages:
        response_data = requests.get(f"{url}?page={page}", headers=headers, params=params, verify=cert_file)
        response_data_json = response_data.json()

        if response_data.status_code != 200:
            print(f"Error: {response_data.status_code}")
            print(response_data.text)  # Error Details
            break

        if total_pages is None:
            total_pages = response_data_json['page']['totalPages']
            print("TOTAL PAGES: ", total_pages)
        
        print(f"PAGE NUMBER: {page}")

        content_entries = response_data_json.get('content', [])
        filtered_results = fn.filter_entries(content_entries, fields)
        all_filtered_results.extend(filtered_results)

        page += 1  # Increase page for the next iteration

    return all_filtered_results

In [6]:
#Funcion to obtain the access token
def get_token_PPDM(instance, username, encrypted_password, cert_file):
    url = f'https://{instance}:8443/api/v2/login'
    headers = {
        'Content-Type': 'application/json'
    }

    # create instance of PasswordManager and decrypt password
    password_manager = PasswordManager()
    password = password_manager.decrypt_password(encrypted_password)

    data = {
        "username": username,
        "password": password
    }

    response = requests.post(url, headers=headers, data=json.dumps(data), verify=cert_file)

    if response.status_code == 200:
        response_json = response.json()
        access_token = response_json.get('access_token')
        refresh_token = response_json.get('refresh_token')
        return access_token, refresh_token
    else:
        print(f"Error: {response.status_code}")
    return None, None

In [7]:
# get health issues of the system
def get_health_issues(instance,access_token, cert_file):
    url = f'https://{instance}:8443/api/v2/system-health-issues'
    headers = {
        'Authorization': access_token
    }
    params = {}  
    fields = [
        "healthCategory",
        "severity",
        "scoreDeduction",
        "componentType",
        "componentName",
        "messageID",
        "detailedDescription",
        "responseAction"
    ]
    return get_filtered_results(url, headers, params, fields, cert_file)

In [8]:
#get list of activities of type "job_group"
def get_job_group_activities(instance, access_token, cert_file, today, twenty_four_hours_ago):
    url = f'https://{instance}:8443/api/v2/activities'
    headers = {
        'Authorization': access_token
    }
    filter_expression = (
        f'createTime ge "{twenty_four_hours_ago}" and createTime lt "{today}" and classType eq "JOB_GROUP"'
    )
    params = {
        'filter': filter_expression
    }
    fields = [
        "category",
        "classType",
        "result.status",
        "createTime",
        "endTime"
    ]
    return get_filtered_results(url, headers, params, fields, cert_file)

In [9]:
# get list of activities with 'status' other than OK and 'protectionPolicy' and 'result.error.code' other than null
def get_activities_not_ok(instance, access_token, cert_file, today, twenty_four_hours_ago):
    url = f'https://{instance}:8443/api/v2/activities'
    headers = {
        'Authorization': access_token
    }
    filter_expression = (
        f'createTime ge "{twenty_four_hours_ago}" and createTime lt "{today}" '
        f'and result.status ne "OK" '
        f'and protectionPolicy.name ne null '
        f'and result.error.code ne null'
    )
    params = {
        'filter': filter_expression
    }
    fields = [
        "category",
        "classType",
        "activityInitiatedType",
        "result.status",
        "result.error.code",
        "result.error.detailedDescription",
        "result.error.extendedReason",
        "result.error.reason",
        "result.error.remediation",
        "asset.name",
        "asset.type",
        "host.name",
        "host.type",
        "inventorySource.type",
        "protectionPolicy.name",
        "protectionPolicy.type",
        "createTime",
        "endTime"
    ]
    return get_filtered_results(url, headers, params, fields, cert_file)

In [10]:
# get storage systems info
def get_storage_systems(instance,access_token, cert_file):
    url = f'https://{instance}:8443/api/v2/storage-systems'
    headers = {
        'Authorization': access_token
    }
    params = {}  
    fields = [
        "type",
        "name",
        "readiness",
        "details"
    ]

    return get_filtered_results(url, headers, params, fields, cert_file)

In [11]:
def main():
    config = fn.load_json_file(config_file)
    base_path = config["basePath"]  # get the basePath from the config file
    json_relative_path = config["jsonPath"]
    jsonPath = os.path.join(base_path, json_relative_path)

    for system, system_data in config["systems"].items():
        json_files = system_data["files"]["json"]  # get the names of the JSON files
        for instance_info in system_data["instances"]:
            instance = instance_info["hostname"]
            username = instance_info["username"]
            encrypted_password = instance_info["encrypted_password"]

            if system == "PPDM":
                cert_relative_path = instance_info["certFile"]
                cert_file = os.path.join(base_path, cert_relative_path)            
                print(system, instance, cert_file)

                # get authentication token
                access_token, _ = get_token_PPDM(instance, username, encrypted_password, cert_file)
                print(access_token)

                if not access_token:
                    print(f"Error: no se pudo obtener el token para {instance}.")
                    continue

                today = get_current_time()
                twenty_four_hours_ago = get_24_hours_ago()

                print("Fetching health issues...")                
                data = get_health_issues(instance, access_token, cert_file)
                fn.save_json(data, system, instance, json_files["systemHealthIssues"], jsonPath)

                print("Fetching job group activities...")
                data = get_job_group_activities(instance, access_token, cert_file, today, twenty_four_hours_ago)
                fn.save_json(data, system, instance, json_files["jobGroupActivitiesSummary"], jsonPath)

                print("Fetching activities that are not OK...")
                data = get_activities_not_ok(instance, access_token, cert_file, today, twenty_four_hours_ago)
                fn.save_json(data, system, instance, json_files["activitiesNotOK"], jsonPath)

                print("Fetching storage systems...")
                data = get_storage_systems(instance, access_token, cert_file)
                fn.save_json(data, system, instance, json_files["storageSystems"], jsonPath)

In [12]:
if __name__ == "__main__":
    main()

PPDM PPDM-01.demo.local E:\DC\dailycheck-PPDM\certs\PPDM_rootca.pem
eyJraWQiOiI4MjQ2MDlkNi00YjE0LTRjYjctOWY5NS01ZTZlYmU0YWRlYmMiLCJhbGciOiJFUzI1NiJ9.eyJqdGkiOiJjNWM0MjM3ZC1kMzFjLTQ2OGMtYmUyMi1lMDk3YjFjZjVkNGYiLCJpc3MiOiJlQ0RNIEFBQSBTZXJ2aWNlIiwiaWF0IjoxNzMzMzA0NjAyLCJleHAiOjE3MzMzMDU1MDIsImV4dCI6eyJzeXN0ZW0iOnsicm9sZSI6WyJyZWFkb25seSJdfSwiYXVkaXQiOnsidWlkIjoiN2U1MTBiM2MtMzZmMC00Yzk0LTk0N2QtNTdjNjhmYjRhMTkwIn19LCJzdWIiOiJEQ09hcGkiLCJhdXRob3JpemF0aW9uLXRva2VuLWJpdG1hcCI6eyJ1c2VybmFtZSI6IkRDT2FwaSIsImF1dGhlbnRpY2F0ZWQiOnRydWUsImlkIjoiN2U1MTBiM2MtMzZmMC00Yzk0LTk0N2QtNTdjNjhmYjRhMTkwIiwidXNlclR5cGUiOiJMT0NBTCIsInRpbWVzdGFtcCI6MCwiY3JlYXRpb25UaW1lIjowLCJ0ZW5hbnRTY29wZSI6Ii8wMDAwMDAwMC0wMDAwLTQwMDAtYjAwMC0wMDAwMDAwMDAwMDAvMDAwMDAwMDAtMDAwMC00MDAwLWEwMDAtMDAwMDAwMDAwMDAwIiwiYXV0aG9yaXRpZXMiOlt7InRlbmFudHMiOlsiMDAwMDAwMDAtMDAwMC00MDAwLWIwMDAtMDAwMDAwMDAwMDAwIl0sInByaXZpbGVnZXMiOlsiLTkwNzg2NTc0MzE2OTIwNzcxNzEiLCIwIl0sInJvbGVzIjpbIjJiYzk4NzUwLTc2NzUtMDEzNi0yOWU5LTViYmQzY2U3MjliMCJdfV0sImdyb3VwcyI